<a href="https://colab.research.google.com/github/Emircankural/116_YazLabProje_2/blob/main/YazLab_Proje_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
from google.colab import files
import xml.etree.ElementTree as ET
import pandas as pd
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    RobertaTokenizer, RobertaForSequenceClassification,
    DebertaTokenizer, DebertaForSequenceClassification,
    AlbertTokenizer, AlbertForSequenceClassification,
    TrainingArguments, Trainer
)
from datasets import Dataset
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# Veri Yükleme
xml_path ="/content/drive/MyDrive/Bütün Tarifler.xml"
tree = ET.parse(xml_path)
root = tree.getroot()

# XML'den DataFrame'e Dönüşüm
data = []
for ingredients in root.findall('Ingredients'):
    malzemeler = [ingredient.text for ingredient in ingredients.findall('Ingredient') if ingredient.text]
    data.append(" ".join(malzemeler))

df = pd.DataFrame(data, columns=['malzemeler'])

# Boş satırları temizle
df = df[df['malzemeler'].str.strip() != ""]

# Etiketleme Fonksiyonu
def etiket_olustur(malzeme_listesi):
    if any(keyword in malzeme_listesi.lower() for keyword in ["yoğurt", "bulgur", "sumak"]):
        return "Turk"
    elif any(keyword in malzeme_listesi.lower() for keyword in ["soy sauce", "tofu", "ginger"]):
        return "Chinese"
    elif any(keyword in malzeme_listesi.lower() for keyword in ["parmesan", "basil", "olive oil"]):
        return "Italian"
    elif any(keyword in malzeme_listesi.lower() for keyword in ["fish and chips", "marmite", "cheddar"]):
        return "British"
    elif any(keyword in malzeme_listesi.lower() for keyword in ["wasabi", "nori", "sake"]):
        return "Japanese"
    return "Other"

df['etiket'] = df['malzemeler'].apply(etiket_olustur)

# Etiketleri sayısal değerlere dönüştürme
df['etiket'] = pd.factorize(df['etiket'])[0]  # Etiketleri numaralandırma

# Boş veri kontrolü
if df.empty:
    raise ValueError("Veri çekme işlemi sonucunda DataFrame boş kaldı. XML dosyasındaki verileri kontrol edin.")

# Tokenizasyon Fonksiyonu
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['malzemeler'], padding='max_length', truncation=True)

# Model ve Tokenizer Seçimi
model_classes = {
    'bert': (BertTokenizer, BertForSequenceClassification, 'bert-base-uncased'),
    'roberta': (RobertaTokenizer, RobertaForSequenceClassification, 'roberta-base'),
    'deberta': (DebertaTokenizer, DebertaForSequenceClassification, 'microsoft/deberta-base'),
    'albert': (AlbertTokenizer, AlbertForSequenceClassification, 'albert-base-v2')
}

for model_name, (TokenizerClass, ModelClass, pretrained_model_name) in model_classes.items():
    print(f"\nModel: {model_name}")

    # Tokenizer ve Model Hazırlığı
    tokenizer = TokenizerClass.from_pretrained(pretrained_model_name)
    model = ModelClass.from_pretrained(pretrained_model_name, num_labels=len(df['etiket'].unique()))

    # Dataset Hazırlığı
    dataset = Dataset.from_pandas(df)
    tokenized_dataset = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_dataset = tokenized_dataset.rename_column('etiket', 'labels')

    # Eğitim ve Test Ayrımı
    train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']

    # Boş veri kontrolü
    if len(train_dataset) == 0 or len(test_dataset) == 0:
        raise ValueError("Eğitim veya test veri seti boş. Etiketleme ve veri bölme işlemini kontrol edin.")

    # Eğitim Argümanları
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs_{model_name}",
        logging_steps=10,
    )

    # Model Eğitimi
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()

    # Model Değerlendirme
    metrics = trainer.evaluate()
    print("Değerlendirme Metrikleri:", metrics)

    # Tahminler ve Metrikler
    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = torch.argmax(torch.tensor(predictions), axis=1)
    print(classification_report(labels, predictions))

    # Karmaşıklık Matrisi
    cm = confusion_matrix(labels, predictions)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title(f"Karmaşıklık Matrisi - {model_name}")
    plt.show()

    # ROC Eğrisi
    fpr, tpr, _ = roc_curve(labels, predictions, pos_label=1)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"ROC curve (area = {roc_auc:.2f})")
    plt.title(f"ROC Eğrisi - {model_name}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.show()



Model: bert


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4310 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
